In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
using PythonCall
agama = pyimport("agama")

In [ ]:
using Printf

In [ ]:
include("agama_utils.jl")
include("agama_plots.jl")

In [ ]:
⊕(a, b) = sqrt(a^2 + b^2)

In [ ]:
using Distributions

In [ ]:
# Nitschai+2021
dist_α_dm = Normal(-1.602, 0.015 ⊕ 0.079)
dist_f_dm = Normal(0.811, 0.006 ⊕ 0.014)
q = 1.3
dist_M_L = Normal(0.413, 0.012 ⊕ 0.031)

In [ ]:
thick_to_thin = 0.12
R_thin = 2.6
R_thick = 3.6
H_thin = 0.3
H_thick = 0.9

In [ ]:
function enc_mass_planar(pot, radius)
    xs = [radius, 0, 0]

    acc = pyconvert(Vector{Float64}, pot.force(xs))
    print(acc)
    a = calc_r(acc)

    return  radius^2 * a
end

In [ ]:
function v_circ_planar(pot, radii)
    xs = [radii zeros(length(radii)) zeros(length(radii))]

    acc = pyconvert(Matrix{Float64}, pot.force(xs))
    a = calc_r(acc')

    return @. sqrt(a * radii)
end

In [ ]:
function plot_v_circ2!(pot; radii=LinRange(4, 30, 1000), kwargs...)
    y = v_circ_planar(pot, radii)
    lines!(radii, y * V2KMS; kwargs...)
end

In [ ]:
v_circ_planar(pots[1], [1,2,3])

In [ ]:
function make_potential(M_L, f_dm, α, q=1.3; verbose=false, r_s=16.8,  Rsun = 8.122, combined=true)
    Φ_thin_unscaled = agama.Potential(type="Disk", surfaceDensity=1, scaleRadius=2.6, scaleHeight=0.3)
    Φ_thick_unscaled = agama.Potential(type="Disk", surfaceDensity=thick_to_thin, scaleRadius=3.6, scaleHeight=0.9)
    Φ_HI = agama.Potential(type="Disk", surfaceDensity=0.00531319, scaleRadius=7, scaleHeight=-0.085, innerCutoffRadius=4)
    Φ_H2 = agama.Potential(type="Disk", surfaceDensity=0.217995, scaleRadius=1.5, scaleHeight=-0.045, innerCutoffRadius=12)

    
    ΣL_disk_solar = 0.056 # lsun, v / kpc^3
    Σ_disk_solar = M_L * ΣL_disk_solar / 10 # to convert to code units

    x_sun = [Rsun, 0, 0]
    
    Σ_disk_unscaled = pyconvert(Float64, Φ_thin_unscaled.density(x_sun) + Φ_thick_unscaled.density(x_sun))
    A_disk = Σ_disk_solar / Σ_disk_unscaled
    ν_bulge = 80.81 # ksun / pc^3
    ρ_bulge = ν_bulge * M_L / 10
    
    
    Φ_bulge = agama.Potential(type="Spheroid", densityNorm=ρ_bulge, axisRatioZ = 0.5, gamma=0, beta=1.8, scaleRadius=0.075, outerCutoffRadius=2.1)
    Φ_thin = agama.Potential(type="Disk", surfaceDensity=A_disk, scaleRadius=2.6, scaleHeight=0.3)
    Φ_thick = agama.Potential(type="Disk", surfaceDensity=A_disk *thick_to_thin, scaleRadius=3.6, scaleHeight=0.9)
    Φ_bary = agama.Potential(Φ_bulge, Φ_thin, Φ_thick, Φ_H2, Φ_HI)
    
    Φ_halo_unscaled = agama.Potential(type="Spheroid", densityNorm=1, alpha=1, beta=3, gamma=-α, scaleRadius=r_s, axisRatioZ=q)
    
    
    Mstar = Φ_bary.enclosedMass(Rsun)
    Mdm = Mstar * f_dm / (1-f_dm)
    Mdm_unscaled = Φ_halo_unscaled.enclosedMass(Rsun)
    A_dm = Mdm / Mdm_unscaled
    Φ_halo = agama.Potential(type="Spheroid", densityNorm=A_dm, alpha=1, beta=3, gamma=-α, scaleRadius=r_s, axisRatioZ=q)
    
    Φ = agama.Potential(Φ_bary, Φ_halo)

    if verbose
        println("Adm = ", A_dm)
        println("A_disk = ", A_disk)
        println("A_bulge = ", ρ_bulge)
        println("Mdm =", Mdm)
        println("Mdm un = ", Mdm_unscaled)
        println("Mstar un = ", Mstar)
    end
    
    if combined
        return Φ
    else
        return Φ_bulge, Φ_thin, Φ_thick, Φ_HI, Φ_H2, Φ_halo
    end
end

In [ ]:
pots[5].enclosedMass(8)

In [ ]:
enc_mass_planar(pots[5], 8)

In [ ]:
pots = make_potential(0.30, 0.86, -1.53, 1.14, r_s=16.8, verbose=true, Rsun=8.122, combined=false)

In [ ]:
pots = make_potential(0.413, 0.811, -1.602, verbose=true, Rsun=8.122, combined=false)

In [ ]:
?plot_v_circ!

In [ ]:
agama.Potential(type="Spheroid", densityNorm=2.3, axisRatioZ = 0.5, gamma=0, beta=1.8, scaleRadius=0.075, outerCutoffRadius=2.1)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], limits=(nothing, nothing, 0, nothing)
    )

kwargs = Dict(
    :log_r =>  log10.(LinRange(1, 100, 1000)),
    :log => true,
    )

plot_v_circ!(pots[1]; label="bulge", kwargs...)
plot_v_circ!(pots[2]; label="thin", kwargs...)
plot_v_circ!(pots[3]; label="thick", kwargs...)
plot_v_circ!(agama.Potential(pots[4], pots[5]); label="gas", kwargs...)
plot_v_circ!(agama.Potential(pots[1:5]...); label="bary", kwargs...)

plot_v_circ!(pots[6]; label="DM", kwargs...)

plot_v_circ!(agama.Potential(pots...); color=:black, label="total", kwargs...)

Legend(fig[1, 2], ax)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 20.5, 0, nothing)
    )

kwargs = Dict(
    :radii =>  (LinRange(0.001, 20, 1000)),
    )

plot_v_circ2!(pots[1]; label="bulge", kwargs...)
plot_v_circ2!(pots[2]; label="thin", kwargs...)
plot_v_circ2!(pots[3]; label="thick", kwargs...)
plot_v_circ2!(agama.Potential(pots[5]); label="gas", kwargs...)
plot_v_circ2!(agama.Potential(pots[1:5]...); label="bary", kwargs...)

plot_v_circ2!(pots[6]; label="DM", kwargs...)
plot_v_circ2!(agama.Potential(pots...); color=:black, label="total", kwargs...)

Legend(fig[1, 2], ax)
fig

In [ ]:
pots[6].density([8.122, 0, 0])

In [ ]:
N = 100
αs = rand(dist_α_dm, N)
fs = rand(dist_f_dm, N)
M_L = rand(dist_M_L, N)

pots = make_potential.(M_L, fs, αs)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    )

kwargs = Dict(
    :alpha=>0.3, 
    )

for pot in pots
    plot_v_circ!(pot, color=:black; kwargs...)


end

fig